<img src="http://www.ubu.es/sites/default/files/portal_page/images/logo_color_2l_dcha.jpg" height="200" width="200" align="right"/> 
### Author: Abel Aioanei 
### Director: César García-Osorio 
### Director: Juan José Rodríguez Díez
### Title: Using my BinaryRelevance classifier and some metrics

### Table of contents:
* [Initializing the classifier](#classifier)
* [Fiting the classifier](#fit)
* [Compute predictions](#predict)
* [Compute probabilities of the predictions](#predict_proba)
* [Compute metrics](#metrics)
* [Feature Selection](#feature_selection)
* [Feature Selection for Label Poverset](#feature_selection_bv)
* [Feature Selection](#feature_selection)
* [Make CrossValidation](#cross)

In [1]:
from library.MyLabelPowerSetFeatureSelect import MyLabelPowerSetFeatureSelect

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import GenericUnivariateSelect

from sklearn.model_selection import train_test_split
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import cross_val_score
from sklearn.metrics import hamming_loss
from sklearn.metrics import accuracy_score

import pandas as pd

Parameters

In [2]:
number_samples= 100
number_classes = 6
number_labels = 2
cross_validation = 5
test_size=0.33
features_to_be_selected = 3

Contructing the Multilabel Dataset

In [3]:
X, y = make_multilabel_classification(n_samples=number_samples, n_classes=number_classes, n_labels=number_classes,sparse = True, allow_unlabeled=False, random_state=1)

Splitting in Training and Test Data

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)


<a id='classifier'></a>
Intializing with the my classifier class

In [5]:
clf = MyLabelPowerSetFeatureSelect()


<a id='fit'></a>
Fit the classifier

In [6]:
clf.fit(X_train, y_train)


<a id='predict'></a>
Compute predictions

In [7]:
predictions = clf.predict(X_test)
pd.DataFrame(predictions.toarray()).head()

,0,1,2,3,4,5
0,1,1,1,1,1,1
1,1,1,0,1,1,0
2,1,1,1,1,1,1
3,1,1,1,1,1,1
4,1,1,0,1,0,0


<a id='predict_proba'></a>
Compute probabilities of the predictions

In [8]:
probability_predictions = clf.predict_proba(X_test)
pd.DataFrame(probability_predictions.toarray()).head()

,0,1,2,3,4,5
0,1.0,1.0,0.501985,1.0,0.894116,0.531109
1,1.0,1.0,0.077886,1.0,0.793062,0.123925
2,1.0,1.0,0.994725,1.0,0.999991,0.999985
3,1.0,1.0,0.944794,1.0,0.998810,0.946245
4,1.0,1.0,0.065435,1.0,0.136636,0.136635


<a id='metrics'></a>
Compute metrics

In [13]:
hamming = hamming_loss(y_test, predictions)
print("Hamming Loss:", "%.3f" % hamming)

accuracy = accuracy_score(y_test, predictions)
print("Accuracy Score:", "%.3f" % accuracy)

Hamming Loss: 0.237
Accuracy Score: 0.152


<a id='feature_selection'></a>
Feature Selection function

In this part I defin a simple function that can do feature selection over a dataset using a given estimator

In [14]:
def feature_select(X, y, estimator):   
        estimator.fit(X, y)
        selected_attributes_indices = estimator.get_support(indices = True)
        
        return selected_attributes_indices#

<a id='estimator'></a>

Select the Feature Selection estimator: SelectKBest or GenericUnivariateSelect

In [11]:
estimator = GenericUnivariateSelect(chi2, 'k_best', param=features_to_be_selected)

In [15]:
estimator = SelectKBest(chi2, k=features_to_be_selected)

Perform the Feature Selection for BR

returns a dataframe with the atributes selected for each instance

In [19]:
list(feature_select(X, y, estimator))

[2, 3, 19]

<a id='cross_validation'></a>
Perform the Cross Validation for the model

In [21]:
scores = cross_val_score(clf, X, y, cv=cross_validation, scoring='accuracy')
average=np.mean(scores)
print(scores)
print(average)

TypeError: Cannot clone object '<library.MyLabelPowerSetFeatureSelect.MyLabelPowerSetFeatureSelect object at 0x00000243C4955D30>' (type <class 'library.MyLabelPowerSetFeatureSelect.MyLabelPowerSetFeatureSelect'>): it does not seem to be a scikit-learn estimator as it does not implement a 'get_params' methods.